# Travelling Salesman Problem using Genetic algorithm
## **By Karthika Nair and Changlong Ji M1-CSN**

In [2]:
import random
import math


# Generate the distance matrix
def generate_map(num_cities):
    distances = [[0] * num_cities for _ in range(num_cities)]
    for i in range(num_cities):
        for j in range(i+1, num_cities):
            # Generate random distances, if two cities are not connected, the distance is set to positive infinity
            distance = random.randint(1, num_cities)
            if random.random() < 0.1:  # 10% probability to set distance to positive infinity
                distance = math.inf
            distances[i][j] = distance
            distances[j][i] = distance  # Set the distance in the symmetric position
    return distances
            
# Output the distance matrix
# distances = generate_map(num_cities = 20)
def print_map(distances):
    for row in distances:
        print(row)


1. Encoding and creating a population

In [3]:
from random import sample

import random


def create_population(population_size = 5,num_cities = 10):
    population = []
    for i in range(population_size):
        new_path = sample(cities, len(cities))
    # If the path already exists in the population, regenerate the path
        while new_path in population:
            new_path = sample(cities, len(cities))
        population.append(new_path)
    return population


In [4]:
# from random import shuffle
# cities = [1,2,3,4,5]
# shuffle(cities)
# print(cities)

Mutation

2. Evaluation:
Calculate the distance of each population

In [5]:
def get_path_distance(path, distances):
    # Initialize the distance to 0
    distance = 0
    # Iterate through each city
    for i in range(num_cities):
        # Current city
        current_city = path[i]
        # Next city (wrap around to first city if necessary)
        next_city = path[(i+1) % num_cities]
        # Add the distance between current city and next city to the total distance
        if next_city == path[0] and i == num_cities - 1:
            distance += distances[current_city-1][next_city-1]
        else:
            distance += distances[current_city-1][next_city-1]
    # Return the total distance
    return distance

def get_distance(population,distances):
    # Initialize list to hold distances for each path in the population
    dis = []
    # Iterate through each path in the population and calculate its distance
    for i in population:
        dis.append(get_path_distance(i, distances))
    # Return the list of distances
    return dis


3. Crossover

In [6]:
def find_parents(population, distances):
    # Create a copy of the population to avoid modifying the original list
    population2 = [i for i in population]
    # Select the first parent as the path with the shortest distance in the population
    parent1 = min(population2, key=lambda x: get_path_distance(x, distances))
    # Remove the first parent from the population
    population2.remove(parent1)
    # Select the second parent as the path with the second shortest distance in the population
    parent2 = min(population2, key=lambda x: get_path_distance(x, distances))
    # Return the two selected parents
    return(parent1, parent2)


In [7]:
import random

def crossover(p1, p2):
    # Make copies of the parents
    a = p1
    b = p2
    # Initialize a list to hold the offspring
    c = [0] * num_cities
    # Select a random starting point from parent 1
    key = random.choice(a)
    # Initialize a list to keep track of visited keys
    key_list = []
    # Iterate through each city in the parents
    j = 0
    while j < len(a):
        i = 0
        while i < len(a):
            # If the key element is found in parent 1, add it to the offspring and update the key
            if a[i] == key:
                c[i] = a[i]
                key_list.append(key)
                key = b[i]
            # If the key is already in the key list, break the loop
            if key in key_list:
                break
            i += 1
        j += 1
    # Replace any remaining 0's in the offspring with cities from parent 2
    for i in c:
        if i == 0:
            c[c.index(i)] = b[c.index(i)]
    # Return the offspring
    return c



4. Mutation

In [8]:
from math import ceil

def mutate(child, num_cities, least=0.2, most=0.5):
    # Select a random index to start the mutation
    mutate_index = random.randint(0, num_cities-1)
    # Select a random number of cities to mutate
    mutate_num = random.randint(ceil(num_cities*least), ceil(num_cities*most))
    # Select a random direction to mutate (left or right)
    random_direction = random.choice([-1, 1])
    # Calculate the start and end indices for the mutation
    mutate_index1 = min(mutate_index, mutate_index+(random_direction)*mutate_num)
    mutate_index2 = max(mutate_index, mutate_index+(random_direction)*mutate_num)
    # Check if the indices are within the range of the city list
    if mutate_index1 < 0:
        mutate_index1 = 0
    if mutate_index2 > num_cities-1:
        mutate_index2 = num_cities-1
    # Reverse the order of the cities within the mutation range
    child[mutate_index1:mutate_index2+1] = reversed(child[mutate_index1:mutate_index2+1])
    # Return the mutated child
    return child



validate if child in population

In [9]:
def validate_child(child, population, num_cities):
    # Check if the child is already in the population, and mutate it if necessary
    for i in range(0, 10):
        if child in population:
            child = mutate(child, num_cities)
        else:
            break
    # Keep mutating the child until it is unique in the population
    while child in population:
        child = mutate(child, num_cities, 0.5, 1)
    # Add the unique child to the population
    population.append(child)
    # Return the updated population
    return population


5. Hamilton

In [10]:
def rotate_array_from_element(arr, element, shift):
    """
    Rotate the array 'arr' from a given 'element' by a given 'shift' amount.
    """
    # Find the index of the element to rotate from
    index = arr.index(element)
    # Get the length of the array
    n = len(arr)
    
    # Rotate the array by shifting the elements after the element to rotate from
    arr = arr[index:] + arr[:index] 
    if shift < 0:
        shift = n + shift 
    arr = arr[shift:] + arr[:shift] 
    
    # Rotate the array back to its original position by shifting the elements before the element to rotate from
    arr = arr[-index:] + arr[:-index] 
    
    # Return the rotated array
    return arr



In [16]:
num_cities = 40
population_size = 15
cities = [i for i in range(1,num_cities+1)]
distances = generate_map(num_cities)
print_map(distances)
population = create_population(population_size)
for i in range(0,200):
    dis = get_distance(population,distances)
    p1,p2 = find_parents(population, distances)
    child = crossover(p1, p2)
    population = validate_child(child,population,num_cities)
    dis = get_distance(population,distances)
    #print(min(dis))
print("min cost:", min(dis))    
#print(dis.index(min(dis))) 
print("min path is", population[dis.index(min(dis))])
p=population[dis.index(min(dis))]
print("From which city u want to start? ")
element=int(input())
shift=p.index(element)
b=rotate_array_from_element(p, element, shift)
b.append(element)
print("the hamilton circuit starting from City",element,"is " ,b )

[0, 25, 4, 32, inf, 4, 17, 2, 32, 24, 29, 33, 23, 6, 35, 1, 6, 20, 5, 23, 39, 24, 6, 36, 7, 35, 12, 40, 38, 8, 38, 34, 16, 18, 30, 11, 31, 15, 3, 16]
[25, 0, 1, 3, 16, 15, inf, 34, 37, inf, 14, 33, inf, 18, 34, 16, 8, 34, 28, 4, 16, inf, 2, inf, 10, 25, 11, 29, 31, 22, 40, 2, 11, 33, 24, inf, inf, 36, 25, 38]
[4, 1, 0, 20, 19, inf, 12, 7, 20, 14, 32, 16, 7, 16, 11, 16, 30, 4, 4, 26, 32, 18, inf, 7, 12, 14, 37, 23, 10, 33, 4, 23, 21, 35, 17, 7, 12, 19, 9, 5]
[32, 3, 20, 0, 6, 5, 14, 16, 20, 23, 15, 6, 13, 5, 37, 20, 10, 6, 10, 5, 33, inf, 35, 25, 31, 5, 39, 19, 23, inf, 1, 27, 39, 34, 18, 5, 40, 6, 16, 19]
[inf, 16, 19, 6, 0, 4, 21, 1, 13, 24, 30, 19, inf, 20, 27, 3, 18, 36, 22, 15, inf, 8, 9, inf, 34, 12, 13, 23, 39, 13, 17, 17, 35, 31, 28, 22, 15, 24, 30, 7]
[4, 15, inf, 5, 4, 0, inf, 25, 4, 7, 3, 28, 9, 17, 19, 38, 14, 39, 11, 25, 13, 36, inf, 13, 38, 1, 30, 14, 8, 33, 12, 3, inf, 38, 5, 30, 7, 18, 24, inf]
[17, inf, 12, 14, 21, inf, 0, inf, 33, 2, inf, 1, 13, 2, 7, 22, 1, 13, inf, 2

Loop and get a local minumal answer

In [17]:
num_cities = 20
population_size = 10
cities = [i for i in range(1,num_cities+1)]
ditsances = generate_map(num_cities)
print_map(distances)
population = create_population(population_size)
for i in range(0,200):
    dis = get_distance(population,distances)
    p1,p2 = find_parents(population, distances)
    child = crossover(p1, p2)
    population = validate_child(child,population,num_cities)
    dis = get_distance(population,distances)
    print(min(dis))
print("min cost:", min(dis))    
print(dis.index(min(dis))) 
print("min path is", population[dis.index(min(dis))])

[0, 25, 4, 32, inf, 4, 17, 2, 32, 24, 29, 33, 23, 6, 35, 1, 6, 20, 5, 23, 39, 24, 6, 36, 7, 35, 12, 40, 38, 8, 38, 34, 16, 18, 30, 11, 31, 15, 3, 16]
[25, 0, 1, 3, 16, 15, inf, 34, 37, inf, 14, 33, inf, 18, 34, 16, 8, 34, 28, 4, 16, inf, 2, inf, 10, 25, 11, 29, 31, 22, 40, 2, 11, 33, 24, inf, inf, 36, 25, 38]
[4, 1, 0, 20, 19, inf, 12, 7, 20, 14, 32, 16, 7, 16, 11, 16, 30, 4, 4, 26, 32, 18, inf, 7, 12, 14, 37, 23, 10, 33, 4, 23, 21, 35, 17, 7, 12, 19, 9, 5]
[32, 3, 20, 0, 6, 5, 14, 16, 20, 23, 15, 6, 13, 5, 37, 20, 10, 6, 10, 5, 33, inf, 35, 25, 31, 5, 39, 19, 23, inf, 1, 27, 39, 34, 18, 5, 40, 6, 16, 19]
[inf, 16, 19, 6, 0, 4, 21, 1, 13, 24, 30, 19, inf, 20, 27, 3, 18, 36, 22, 15, inf, 8, 9, inf, 34, 12, 13, 23, 39, 13, 17, 17, 35, 31, 28, 22, 15, 24, 30, 7]
[4, 15, inf, 5, 4, 0, inf, 25, 4, 7, 3, 28, 9, 17, 19, 38, 14, 39, 11, 25, 13, 36, inf, 13, 38, 1, 30, 14, 8, 33, 12, 3, inf, 38, 5, 30, 7, 18, 24, inf]
[17, inf, 12, 14, 21, inf, 0, inf, 33, 2, inf, 1, 13, 2, 7, 22, 1, 13, inf, 2